In [ ]:
from google.colab import drive  # 구글 코랩에서 drive(내 구글 드라이브)를 사용하기 위한 함수
import io
drive.mount('/content/drive')  # 이렇게 하면 내 drive가 설정? 잡힌다.

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8MB 340kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670911 sha256=b2b635063ae2ec83694872a8c07555616ff1aa367fc190b85a1e42ee01d851ac
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
import pandas as pd # pandas 불러오기
import numpy as np # numpy 불러오기
import math # 불러오기 수학과 관련된 함수들
import re #정규표현식 함수 .맞는지 틀린지
from scipy.sparse import csr_matrix   # 매트릭스 해주는거.
import matplotlib.pyplot as plt # 시각화 함수
import seaborn as sns # 시각화 함수
from surprise import Reader, Dataset, SVD, NormalPredictor, KNNBasic
from surprise import KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise import BaselineOnly, SVDpp,NMF, SlopeOne, CoClustering #분석툴
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import cross_validate, KFold, train_test_split
import json
import mpmath
from scipy.special import logsumexp

from google.colab import drive


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
ratings = pd.read_csv('../content/drive/My Drive/data/the-movies-dataset/ratings_small.csv',low_memory=False)
ratings = ratings[['userId', 'movieId', 'rating']]
ratings.head()
ratings.movieId = pd.to_numeric(ratings.movieId, errors='coerce')
ratings.userId = pd.to_numeric(ratings.userId, errors='coerce')
ratings.rating = pd.to_numeric(ratings.rating, errors='coerce')
len(ratings)
df = ratings
print(df)

        userId  movieId  rating
0            1       31     2.5
1            1     1029     3.0
2            1     1061     3.0
3            1     1129     2.0
4            1     1172     4.0
...        ...      ...     ...
99999      671     6268     2.5
100000     671     6269     4.0
100001     671     6365     4.0
100002     671     6385     2.5
100003     671     6565     3.5

[100004 rows x 3 columns]


In [ ]:
p = df.groupby('rating')['rating'].agg(['count'])
movie_count = df['movieId'].nunique()
cust_count = df['userId'].nunique()
rating_count = df['userId'].count()
df = df[pd.notnull(df['rating'])]

In [ ]:
f = ['count','mean']
df_movie_summary = df.groupby('movieId')['rating'].agg(f)
df_movie_summary.index = df_movie_summary.index.map(int) # map 함수 쓰면 한번에 형변환 처리 가능 , 스트나 튜플을 지정함수로 처리해주는 역할
movie_benchmark = round(df_movie_summary['count'].quantile(0.7),0)  #quantile 사분위수
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index
print('Movie minimum times of review: {}'.format(movie_benchmark))

Movie minimum times of review: 7.0


In [ ]:
df_cust_summary = df.groupby('userId')['rating'].agg(f)
df_cust_summary.index = df_cust_summary.index.map(int)
cust_benchmark = round(df_cust_summary['count'].quantile(0.7),0)
drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_benchmark].index
print('Customer minimum times of review: {}'.format(cust_benchmark))

Customer minimum times of review: 138.0


In [ ]:
print('Original Shape: {}'.format(df.shape))
df = df[~df['movieId'].isin(drop_movie_list)] # df의 'Movie_Id'에서 drop_movie_list의 값이 있으면 True
print('df not isin dropmovie', df)
df = df[~df['userId'].isin(drop_cust_list)]
print('df not isin dropcust', df)
print('After Trim Shape: {}'.format(df.shape))
print('-Data Examples-')
print(df.iloc[::5000000, :])

Original Shape: (100004, 3)
df not isin dropmovie         userId  movieId  rating
0            1       31     2.5
1            1     1029     3.0
2            1     1061     3.0
3            1     1129     2.0
4            1     1172     4.0
...        ...      ...     ...
99998      671     6212     2.5
100000     671     6269     4.0
100001     671     6365     4.0
100002     671     6385     2.5
100003     671     6565     3.5

[86257 rows x 3 columns]
df not isin dropcust        userId  movieId  rating
147         4       10     4.0
148         4       34     5.0
149         4      112     5.0
150         4      141     5.0
151         4      153     4.0
...       ...      ...     ...
99686     665     5479     2.0
99688     665     5502     4.0
99689     665     5679     3.0
99690     665     5952     5.0
99691     665     5991     4.0

[61805 rows x 3 columns]
After Trim Shape: (61805, 3)
-Data Examples-
     userId  movieId  rating
147       4       10     4.0


In [ ]:
df_p = pd.pivot_table(df,values='rating',index='userId',columns='movieId')
print(df_p)

movieId  1       2       3       4       ...  139644  142488  148626  152081
userId                                   ...                                
4           NaN     NaN     NaN     NaN  ...     NaN     NaN     NaN     NaN
15          2.0     2.0     NaN     NaN  ...     3.0     3.5     3.5     3.0
17          NaN     NaN     NaN     NaN  ...     NaN     NaN     NaN     NaN
19          3.0     3.0     3.0     3.0  ...     NaN     NaN     NaN     NaN
21          NaN     NaN     NaN     NaN  ...     NaN     NaN     NaN     NaN
...         ...     ...     ...     ...  ...     ...     ...     ...     ...
652         NaN     NaN     NaN     NaN  ...     NaN     NaN     NaN     NaN
654         5.0     3.0     NaN     NaN  ...     NaN     NaN     NaN     NaN
659         NaN     NaN     NaN     NaN  ...     NaN     NaN     NaN     NaN
664         3.5     NaN     NaN     NaN  ...     NaN     NaN     NaN     NaN
665         NaN     3.0     3.0     NaN  ...     NaN     NaN     NaN     NaN

In [ ]:
meta = pd.read_csv('../content/drive/My Drive/data/the-movies-dataset/movies_metadata.csv',low_memory=False)


In [ ]:
meta.head(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.924927,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,6.677277,/jQh15y5YB7bWz1NtffNZmRw0s9D.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",2.561161,/sGO5Qa55p7wTu7FJcX4H4xIVKvS.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,To

In [ ]:
meta = meta[['id', 'original_title', 'genres','release_date','popularity','original_language']]
meta = meta.rename(columns={'id':'movieId'})
meta.movieId = pd.to_numeric(meta.movieId, errors='coerce')
meta.popularity = pd.to_numeric(meta.popularity, errors='coerce')   # popularity를 문자열에서 숫자형으로 변환!

meta.head()

,movieId,original_title,genres,release_date,popularity,original_language
0,862.0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995-10-30,21.946943,en
1,8844.0,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995-12-15,17.015539,en
2,15602.0,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995-12-22,11.712900,en
3,31357.0,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995-12-22,3.859495,en
4,11862.0,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]",1995-02-10,8.387519,en


In [ ]:
reader = Reader()
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
svd = SVD()
cross_validate(svd, data, cv=5 , measures=['RMSE', 'MAE']) #evaluate 대신 cross_validate사용
# cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


{'fit_time': (3.077868938446045,
  3.066119432449341,
  3.1023316383361816,
  3.1070029735565186,
  3.1060357093811035),
 'test_mae': array([0.66344649, 0.66694699, 0.66097618, 0.6729848 , 0.67496686]),
 'test_rmse': array([0.86275001, 0.86803381, 0.86238156, 0.8729458 , 0.87417363]),
 'test_time': (0.09153914451599121,
  0.08650374412536621,
  0.1507730484008789,
  0.08809685707092285,
  0.09121561050415039)}

In [ ]:
def userRec(data,usernumber,rating,moviedata,dropdata,reader,svd):
    df = data
    df_user = df[(df['userId'] == usernumber) & (df['rating'] == rating)]
    df_user = df_user.set_index('movieId')
    df_user = df_user.join(moviedata)['original_title']
    print(df_user)
    user_df = moviedata.copy()
    user_df = user_df[~user_df['movieId'].isin(dropdata)]
    data1 = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
    trainset = data1.build_full_trainset()
    svd.fit(trainset)
    user_df['Estimate_Score'] = user_df['movieId'].apply(lambda x: svd.predict(usernumber, x).est)
    user_df = user_df.drop('movieId', axis = 1)
    user_df = user_df.sort_values('Estimate_Score', ascending=False)
    print(user_df.head(10))
userRec(df,665,5,meta,drop_movie_list,reader,svd)

movieId
168                                       Jeffrey
317                             Fresa y chocolate
318                          Swimming with Sharks
364            Mrs. Parker and the Vicious Circle
527                                    Serial Mom
587               Snow White and the Seven Dwarfs
595                         A Great Day in Harlem
596                                  Bye Bye Love
597                                  One Fine Day
912                       The Reluctant Debutante
1073       Children of the Corn IV: The Gathering
1097                            Manon des Sources
1101                                Hustler White
1136                   La double vie de Véronique
1270                            I'm Not Rappaport
1704                                  Wild Things
1754                                   Species II
2081                    Why Do Fools Fall In Love
2087                                    Knock Off
2125                                  Simo

In [ ]:
#여기까지 user추천이었습니다.
#아래부터는 영화제목 추천입니다.

In [ ]:
def parse_genres(genres_str):
    genres = json.loads(genres_str.replace('\'', '"'))
    
    genres_list = []
    for g in genres:
        genres_list.append(g['name'])

    return genres_list

meta['genres'] = meta['genres'].apply(parse_genres) #각 행에대해 함수를 적용하려면 apply를 써야 된다.

In [ ]:
data = pd.merge(ratings, meta, on='movieId', how='inner')
data.head()

,userId,movieId,rating,original_title,genres,release_date,popularity,original_language
0,1,1371,2.5,Rocky III,[Drama],1982-05-28,11.673366,en
1,4,1371,4.0,Rocky III,[Drama],1982-05-28,11.673366,en
2,7,1371,3.0,Rocky III,[Drama],1982-05-28,11.673366,en
3,19,1371,4.0,Rocky III,[Drama],1982-05-28,11.673366,en
4,21,1371,3.0,Rocky III,[Drama],1982-05-28,11.673366,en


In [ ]:
matrix = data.pivot_table(index='userId', columns='original_title', values='rating')
matrix.head(20)

original_title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,...Più forte ragazzi!,00 Schneider - Jagd auf Nihil Baxter,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 + 1,12 Angry Men,13 Tzameti,1408,15 Minutes,16 Blocks,1724 기방난동사건,1942: A Love Story,1984,2 Days in Paris,"20,000 Leagues Under the Sea",2001: A Space Odyssey,2010,2046,2061 - Un anno eccezionale,21 Grams,24 Hour Party People,25th Hour,28 Days Later,28 Weeks Later,29th Street,2:37,3 Ninjas: High Noon at Mega Mountain,3 hommes et un couffin,300,33 sceny z życia,36 Fillette,"4 luni, 3 săptămîni și 2 zile",40 Days and 40 Nights,42nd Street,48 Hrs.,...,真夜中の弥次さん喜多さん,続宮本武蔵 一乗寺の決闘,綠草地,纵横四海,羅生門,英雄,菊次郎の夏,薔薇の葬列,蜘蛛巣城,裸の島,誰も知らない,豚と軍艦,赤ひげ,野火,鎗王之王,隠し砦の三悪人,風の谷のナウシカ,香港製造,鬼婆,鷹爪鐵布衫,거룩한 계보,고지전,공동경비구역 JSA,괴물,김복남 살인사건의 전말,다세포 소녀,도쿄!,밀양,봄 여름 가을 겨울 그리고 봄,빈집,사마리아,야수,연가시,오직 그대만,올드보이,"장화, 홍련",최종병기 활,친절한 금자씨,해안선,활
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
GENRE_WEIGHT = 0.1
def pearsonR(s1, s2):  # 피어슨 상관관계를 만드는 함수
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.exp(logsumexp(np.sum(s1_c * s2_c))) / np.exp(logsumexp(np.sqrt(np.sum(s1_c ** 2)))) * np.exp(logsumexp(np.sum(s2_c ** 2)))
    # 분자가 클수록 상관관계가 높다.
        # 유저가 넣은 영화 / 피벗테이블 /반환값 /장르별 가중치를 할지 선택
def recommend(input_movie, matrix, n, similar_genre=True):
    input_genres = meta[meta['original_title'] == input_movie]['genres'].iloc(0)[0]
    # 값을 넣은 movie와 meta가 같을 때  origin_title의 장르를 가져온다.
    result = []
    for title in matrix.columns:
        if title == input_movie: # 타이틀과 input_movie가 같으면 넘긴다.
            continue

        # rating comparison
        # 매트릭스의 컬럼이름으로 루프를 돈다. title은 그 제목이고
        # 여기서 매트릭스의 타이틀이름과 inputmovice 를 넣고 피어슨 상관관계를 본다.
        cor = pearsonR(matrix[input_movie], matrix[title]) #피어슨 선형관계
        #여기서 cor 값을 구하고 

        # genre comparison (장르별 가중치를 두는 작업)
        if similar_genre and len(input_genres) > 0: # 장르별 가중치가 있고 장르가 있을 경우
            #루프 돌고있는 matirx의 컬럼명 값이 영화 제목을 만나면 장르값을 저장한다.
            temp_genres = meta[meta['original_title'] == title]['genres'].iloc(0)[0]
            # np.sum으로 리스트에 있는 값들을 합친다?
            # 찾는 값이 있는지에 따라 true false를 반환한다. 왼쪽에 오른쪽 값이 있는지 확인한다.
            # 그에따라 true 개수를 세서 같은게 몇개있는지 분석한다.
            same_count = np.sum(np.isin(input_genres, temp_genres))

            #선형관계에서 나온 계수에 장르별 가중치를 더해준다.
            cor += (GENRE_WEIGHT * same_count)
        
        if np.isnan(cor): # 값이 없으면 넘기고
            continue
        else: #값이 있으면 result라는 리스트에 값을 넣는다.
            result.append((title, '{:.2f}'.format(cor), temp_genres))
    # 정렬해준다.
    result.sort(key=lambda r: r[1], reverse=True)

    return result[:n]

In [ ]:

recommend_result = recommend('The Dark Knight', matrix, 10, similar_genre=True)

pd.DataFrame(recommend_result, columns = ['Title', 'Correlation', 'Genre'])

,Title,Correlation,Genre
0,Mad Max 2,9994640.90,"[Adventure, Action, Thriller, Science Fiction]"
1,Guter Junge,99411842.15,[Drama]
2,Bonnie and Clyde,993379.48,"[Crime, Drama]"
3,Ghost Dog: The Way of the Samurai,991423897390230542483456.00,"[Crime, Drama]"
4,纵横四海,9906.27,"[Action, Comedy, Thriller, Crime]"
5,Finding Nemo,990022858.27,"[Animation, Family]"
6,Dreamgirls,987590976315022.38,[Drama]
7,Breaking and Entering,9856785.89,"[Drama, Romance]"
8,Yesterday,9839656214640357015552.00,[Drama]
9,Beowulf,982755.20,"[Action, Science Fiction, Thriller, Adventure,..."


# 여기 까지가 기존 소스 아래부터는 변수 추가 부분과 알고리즘

In [ ]:
def user_release_ratio(df, usernumber):
    user_df =df[df['userId'] == usernumber]
    meta2 = pd.read_csv('../content/drive/My Drive/data/the-movies-dataset/movies_metadata.csv',low_memory=False)
    value_meta = meta2[['id','original_title','release_date', 'genres']]

    value_meta = value_meta.rename(columns={'id':'movieId'})
    value_meta.movieId = pd.to_numeric(value_meta.movieId, errors='coerce')
    value_meta = value_meta.dropna(axis=0)
    value_meta = value_meta.reset_index()
    merge_data = pd.merge(user_df, value_meta, on='movieId', how='left')
    merge_data = merge_data.dropna(axis=0)
    merge_data = merge_data.reset_index()

    release_date_list = {'1900':0,'1950':0,'1960':0,'1970':0,'1980':0,'1990':0,'2000':0,'2010':0,'2020':0}
    for i in range(0,len(merge_data)):
        if int(merge_data['release_date'].loc[i][0:4]) <= 1900:
            release_date_list["1900"] += 1
        elif int(merge_data['release_date'].loc[i][0:4]) <= 1950:
            release_date_list["1950"] += 1
        elif int(merge_data['release_date'].loc[i][0:4]) <= 1960:
            release_date_list["1960"] += 1
        elif int(merge_data['release_date'].loc[i][0:4]) <= 1970:
            release_date_list["1970"] += 1
        elif int(merge_data['release_date'].loc[i][0:4]) <= 1980:
            release_date_list["1980"] += 1
        elif int(merge_data['release_date'].loc[i][0:4]) <= 1990:
            release_date_list["1990"] += 1
        elif int(merge_data['release_date'].loc[i][0:4]) <= 2000:
            release_date_list["2000"] += 1
        elif int(merge_data['release_date'].loc[i][0:4]) <= 2010:
            release_date_list["2010"] += 1
        elif int(merge_data['release_date'].loc[i][0:4]) <= 2020:
            release_date_list["2020"] += 1
    release_date_list

    sum = 0
    for i in release_date_list:

        sum += release_date_list[i]

    release_date_rate = []
    for i in release_date_list:
        if release_date_list[i] ==0:
            continue
        release_date_list[i] = round((release_date_list[i]/sum),3)
    return release_date_list

In [ ]:
def Estimate_Score_sum(user_df, user_release_ratio_list):
    user_df = user_df.dropna(axis=0)
    for i in range(0,len(user_df)):
        if int(user_df.iloc[i]['release_date'][0:4]) <= 1900:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_release_ratio_list['1900']
        elif int(user_df.iloc[i]['release_date'][0:4]) <= 1950:
            user_df['Estimate_Score'].loc[user_df.index[i]]= user_df.iloc[i]['Estimate_Score'] + user_release_ratio_list['1950']
        elif int(user_df.iloc[i]['release_date'][0:4]) <= 1960:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_release_ratio_list['1960']
        elif int(user_df.iloc[i]['release_date'][0:4]) <= 1970:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_release_ratio_list['1970']
        elif int(user_df.iloc[i]['release_date'][0:4]) <= 1980:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_release_ratio_list['1980']
        elif int(user_df.iloc[i]['release_date'][0:4]) <= 1990:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_release_ratio_list['1990']
        elif int(user_df.iloc[i]['release_date'][0:4])  <= 2000:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_release_ratio_list['2000']
        elif int(user_df.iloc[i]['release_date'][0:4])  <= 2010:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_release_ratio_list['2010']
        elif int(user_df.iloc[i]['release_date'][0:4])  <= 2020:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_release_ratio_list['2020']
    return user_df


In [ ]:
def userRec2(data,usernumber,rating,moviedata,dropdata,reader,svd):
    df = data
    df_user = df[(df['userId'] == usernumber) & (df['rating'] == rating)]
    df_user = df_user.set_index('movieId')
    df_user = df_user.join(moviedata)['original_title']
    print(df_user)

    user_release_ratio_list = user_release_ratio(df, usernumber) # 유저의 년도 비율을 가져온다.

    user_df = moviedata.copy()
    user_df = user_df[~user_df['movieId'].isin(dropdata)]
    data1 = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
    trainset = data1.build_full_trainset()
    svd.fit(trainset)
    user_df['Estimate_Score'] = user_df['movieId'].apply(lambda x: svd.predict(usernumber, x).est)
    user_df = user_df.drop('movieId', axis = 1)
    user_df = user_df.sort_values('Estimate_Score', ascending=False)
    print(user_df.head(10))

    user_df_sum = Estimate_Score_sum(user_df, user_release_ratio_list)
    user_df_sum = user_df_sum.sort_values('Estimate_Score', ascending=False)
    print(user_df_sum.head(10))
    return user_df, user_release_ratio_list
test,test2 = userRec2(df,665,5,meta,drop_movie_list,reader,svd)

movieId
168                                       Jeffrey
317                             Fresa y chocolate
318                          Swimming with Sharks
364            Mrs. Parker and the Vicious Circle
527                                    Serial Mom
587               Snow White and the Seven Dwarfs
595                         A Great Day in Harlem
596                                  Bye Bye Love
597                                  One Fine Day
912                       The Reluctant Debutante
1073       Children of the Corn IV: The Gathering
1097                            Manon des Sources
1101                                Hustler White
1136                   La double vie de Véronique
1270                            I'm Not Rappaport
1704                                  Wild Things
1754                                   Species II
2081                    Why Do Fools Fall In Love
2087                                    Knock Off
2125                                  Simo

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: 

                       original_title  ... Estimate_Score
11922                  License to Wed  ...       4.949300
534              Sleepless in Seattle  ...       4.848254
286                Once Were Warriors  ...       4.764541
4886                The Shipping News  ...       4.697719
11410                10 Items or Less  ...       4.696637
7727                              座頭市  ...       4.633283
10089                      Madagascar  ...       4.622506
938                      The 39 Steps  ...       4.617667
2648          The Thomas Crown Affair  ...       4.589246
18256  Je vais bien, ne t'en fais pas  ...       4.579948

[10 rows x 6 columns]


In [ ]:
df

,userId,movieId,rating
147,4,10,4.0
148,4,34,5.0
149,4,112,5.0
150,4,141,5.0
151,4,153,4.0
...,...,...,...
99686,665,5479,2.0
99688,665,5502,4.0
99689,665,5679,3.0
99690,665,5952,5.0


In [ ]:
def userRec3(data,usernumber,rating,moviedata,dropdata,reader,algo):
    df = data
    df_user = df[(df['userId'] == usernumber) & (df['rating'] == rating)]
    df_user = df_user.set_index('movieId')
    df_user = df_user.join(moviedata)['original_title']
    print(df_user)

    user_release_ratio_list = user_release_ratio(df, usernumber) # 유저의 년도 비율을 가져온다.

    user_df = moviedata.copy()
    user_df = user_df[~user_df['movieId'].isin(dropdata)]
    data1 = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
    trainset = data1.build_full_trainset()
    algo.fit(trainset)
    user_df['Estimate_Score'] = user_df['movieId'].apply(lambda x: algo.predict(usernumber, x).est)
    user_df = user_df.drop('movieId', axis = 1)
    user_df = user_df.sort_values('Estimate_Score', ascending=False)
    print(user_df.head(10))

    user_df_sum = Estimate_Score_sum(user_df, user_release_ratio_list)
    user_df_sum = user_df_sum.sort_values('Estimate_Score', ascending=False)
    print(user_df_sum.head(10))
    return user_df, user_release_ratio_list
test,test2 = userRec2(df,665,5,meta,drop_movie_list,reader,svd)

movieId
168                                       Jeffrey
317                             Fresa y chocolate
318                          Swimming with Sharks
364            Mrs. Parker and the Vicious Circle
527                                    Serial Mom
587               Snow White and the Seven Dwarfs
595                         A Great Day in Harlem
596                                  Bye Bye Love
597                                  One Fine Day
912                       The Reluctant Debutante
1073       Children of the Corn IV: The Gathering
1097                            Manon des Sources
1101                                Hustler White
1136                   La double vie de Véronique
1270                            I'm Not Rappaport
1704                                  Wild Things
1754                                   Species II
2081                    Why Do Fools Fall In Love
2087                                    Knock Off
2125                                  Simo

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/l

                 original_title  ... Estimate_Score
4020   The Million Dollar Hotel  ...       4.831255
286          Once Were Warriors  ...       4.746239
11355      Flags of Our Fathers  ...       4.719946
5295                5 Card Stud  ...       4.708241
5253                     Enough  ...       4.699997
11152                     Crank  ...       4.654071
5284        The Bourne Identity  ...       4.653576
534        Sleepless in Seattle  ...       4.650367
9507           Muxmäuschenstill  ...       4.627008
14332                    Аэлита  ...       4.615243

[10 rows x 6 columns]


In [ ]:
def user_pop_ratio(df, usernumber):
    user_df =df[df['userId'] == usernumber]
    meta2 = pd.read_csv('../content/drive/My Drive/data/the-movies-dataset/movies_metadata.csv',low_memory=False)
    value_meta = meta2[['id','original_title', 'popularity','genres']]

    value_meta = value_meta.rename(columns={'id':'movieId'})
    value_meta.movieId = pd.to_numeric(value_meta.movieId, errors='coerce')
    value_meta.popularity = pd.to_numeric(value_meta.popularity, errors='coerce') 
    value_meta = value_meta.dropna(axis=0)
    value_meta = value_meta.reset_index()
    merge_data = pd.merge(user_df, value_meta, on='movieId', how='left')
    merge_data = merge_data.dropna(axis=0)
    merge_data = merge_data.reset_index()

    popularity_list = {'2':0,'6':0,'9':0,'13':0,'18':0,'31':0,'64':0,'185':0,'288':0, '547':0}
    for i in range(0,len(merge_data)):
        if int(merge_data['popularity'].loc[i]) <= 2:
            popularity_list["2"] += 1
        elif int(merge_data['popularity'].loc[i]) <= 6:
            popularity_list["6"] += 1
        elif int(merge_data['popularity'].loc[i]) <= 9:
            popularity_list["9"] += 1
        elif int(merge_data['popularity'].loc[i]) <= 13:
            popularity_list["13"] += 1
        elif int(merge_data['popularity'].loc[i]) <= 18:
            popularity_list["18"] += 1
        elif int(merge_data['popularity'].loc[i]) <= 31:
            popularity_list["31"] += 1
        elif int(merge_data['popularity'].loc[i]) <= 64:
            popularity_list["64"] += 1
        elif int(merge_data['popularity'].loc[i]) <= 185:
            popularity_list["185"] += 1
        elif int(merge_data['popularity'].loc[i]) <= 288:
            popularity_list["288"] += 1
        elif int(merge_data['popularity'].loc[i]) <= 547:
            popularity_list["547"] += 1
    popularity_list

    sum = 0
    for i in popularity_list:

        sum += popularity_list[i]

    #popularity_list = []
    for i in popularity_list:
        if popularity_list[i] ==0:
            continue
        popularity_list[i] = round((popularity_list[i]/sum),3)
    return popularity_list

In [ ]:
def Estimate_Score_sum2(user_df, user_pop_ratio_list):
    user_df = user_df.dropna(axis=0)
    for i in range(0,len(user_df)):
        if int(user_df.iloc[i]['popularity']) <= 2:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['2']
        elif int(user_df.iloc[i]['popularity']) <= 6:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['6']
        elif int(user_df.iloc[i]['popularity']) <= 9:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['9']
        elif int(user_df.iloc[i]['popularity']) <= 13:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['13']
        elif int(user_df.iloc[i]['popularity']) <= 18:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['18']
        elif int(user_df.iloc[i]['popularity']) <= 31:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['31']
        elif int(user_df.iloc[i]['popularity'])  <= 64:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['64']
        elif int(user_df.iloc[i]['popularity'])  <= 185:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['185']
        elif int(user_df.iloc[i]['popularity'])  <= 288:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['288']
        elif int(user_df.iloc[i]['popularity'])  <= 547:
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_pop_ratio_list['547']
    return user_df


In [ ]:
def user_language_ratio(df, usernumber):
    user_df =df[df['userId'] == usernumber]
    meta2 = pd.read_csv('../content/drive/My Drive/data/the-movies-dataset/movies_metadata.csv',low_memory=False)
    value_meta = meta2[['id','original_title', 'original_language','genres']]

    value_meta = value_meta.rename(columns={'id':'movieId'})
    value_meta.movieId = pd.to_numeric(value_meta.movieId, errors='coerce')
    # value_meta.popularity = pd.to_numeric(value_meta.popularity, errors='coerce') 
    value_meta = value_meta.dropna(axis=0)
    value_meta = value_meta.reset_index()
    merge_data = pd.merge(user_df, value_meta, on='movieId', how='left')
    merge_data = merge_data.dropna(axis=0)
    merge_data = merge_data.reset_index()

    original_language_list = {'en':0,'fr':0,'it':0,'ja':0,'de':0}
    for i in range(0,len(merge_data)):
        if merge_data['original_language'].loc[i] == 'en':
            original_language_list["en"] += 1
        elif merge_data['original_language'].loc[i] == 'fr':
            original_language_list["fr"] += 1
        elif merge_data['original_language'].loc[i] == 'it':
            original_language_list["it"] += 1
        elif merge_data['original_language'].loc[i] == 'ja':
            original_language_list["ja"] += 1
        elif merge_data['original_language'].loc[i] == 'de':
            original_language_list["de"] += 1
       
    original_language_list
    print("***wow1**")
    print(original_language_list)
    print("***wow1**")

    sum = 0
    for i in original_language_list:

        sum += original_language_list[i]

    # popularity_list = []
    for i in original_language_list:
        if original_language_list[i] ==0:
            continue
        original_language_list[i] = round((original_language_list[i]/sum),3)
    print("***wow2**")
    print(original_language_list)
    print("***wow2**")

    return original_language_list

In [ ]:
def Estimate_Score_sum3(user_df, user_language_ratio_list):
    user_df = user_df.dropna(axis=0)
    for i in range(0,len(user_df)):
        if user_df.iloc[i]['original_language'] == 'en':
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_language_ratio_list['en']
        elif user_df.iloc[i]['original_language'] == 'fr':
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_language_ratio_list['fr']
        elif user_df.iloc[i]['original_language'] == 'it':
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_language_ratio_list['it']
        elif user_df.iloc[i]['original_language'] == 'ja':
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_language_ratio_list['ja']
        elif user_df.iloc[i]['original_language'] == 'de':
            user_df['Estimate_Score'].loc[user_df.index[i]] = user_df.iloc[i]['Estimate_Score'] + user_language_ratio_list['de']
       
    return user_df

In [ ]:
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating', ]], reader)
kf = KFold(n_splits=3)
kf.split(data)

slope = SlopeOne()
cross_validate(slope, data, measures=['RMSE', 'MAE'],cv=5, verbose=False) #evaluate 대신 cross_validate


{'fit_time': (1.2666945457458496,
  1.306401014328003,
  1.2620127201080322,
  1.2635891437530518,
  1.2987513542175293),
 'test_mae': array([0.67587805, 0.67728643, 0.67564195, 0.66272236, 0.67387141]),
 'test_rmse': array([0.88296564, 0.87711025, 0.88023342, 0.86030705, 0.88023473]),
 'test_time': (4.173664569854736,
  4.1946797370910645,
  4.224207401275635,
  4.266580104827881,
  4.183549404144287)}

In [ ]:
reader = Reader()
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
kf = KFold(n_splits=3)
kf.split(data)
svd = SVD()
cross_validate(svd, data, cv = 5,  measures=['RMSE', 'MAE']) #evaluate 대신 cross_validate사용

{'fit_time': (3.130192756652832,
  3.136723041534424,
  3.1538171768188477,
  3.1499526500701904,
  3.1476070880889893),
 'test_mae': array([0.66716751, 0.67121281, 0.66873132, 0.65918521, 0.66351617]),
 'test_rmse': array([0.86841381, 0.8734712 , 0.87006273, 0.85658828, 0.86057403]),
 'test_time': (0.08469653129577637,
  0.08692526817321777,
  0.08544778823852539,
  0.08823537826538086,
  0.08668828010559082)}

In [ ]:
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
kf = KFold(n_splits=3)
kf.split(data)

option = {'name' : 'pearson'}
knn = KNNBasic(sim_options=option)
cross_validate(knn, data, cv = 5 , measures=['RMSE', 'MAE']) #evaluate 대신 cross_validate사용

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


{'fit_time': (0.0916292667388916,
  0.10385465621948242,
  0.1067054271697998,
  0.10507011413574219,
  0.10706615447998047),
 'test_mae': array([0.73899366, 0.73816978, 0.74955232, 0.73143539, 0.73921064]),
 'test_rmse': array([0.94939854, 0.94445354, 0.96253114, 0.93598404, 0.94658404]),
 'test_time': (0.7806992530822754,
  0.8803696632385254,
  0.7623260021209717,
  0.7743539810180664,
  0.7658829689025879)}

In [ ]:
reader = Reader()
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
kf = KFold(n_splits=3)
kf.split(data)
# svd = SVD()
# cross_validate(svd, data, measures=['RMSE', 'MAE']) #evaluate 대신 cross_validate사용

nmf = NMF()
cross_validate(nmf, data, measures=['RMSE', 'MAE'],cv=3, verbose=False) #evaluate 대신 cross_validate


{'fit_time': (2.781510829925537, 2.753741979598999, 2.8106203079223633),
 'test_mae': array([0.70470134, 0.70400364, 0.70190669]),
 'test_rmse': array([0.91703062, 0.91420598, 0.91320865]),
 'test_time': (0.22315239906311035, 0.12439846992492676, 0.12685441970825195)}

In [ ]:
def userRec3(data,usernumber,rating,moviedata,dropdata,reader,algo):
    df = data
    df_user = df[(df['userId'] == usernumber) & (df['rating'] == rating)]
    df_user = df_user.set_index('movieId')
    df_user = df_user.join(moviedata)['original_title']
    # print(df_user)

    user_release_ratio_list = user_release_ratio(df, usernumber) # 유저의 년도 비율을 가져온다.
    print(user_release_ratio_list)
    user_pop_ratio_list = user_pop_ratio(df, usernumber) # 유저의 popularity 비율을 가져온다.
    print(user_pop_ratio_list)
    user_language_ratio_list = user_language_ratio(df, usernumber) # 유저의 language 비율을 가져온다.
    print(user_language_ratio_list)

    user_df = moviedata.copy()
    user_df = user_df[~user_df['movieId'].isin(dropdata)]
    data1 = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
    trainset = data1.build_full_trainset()
    algo.fit(trainset)
    user_df['Estimate_Score'] = user_df['movieId'].apply(lambda x: algo.predict(usernumber, x).est)
    user_df = user_df.drop('movieId', axis = 1)
    user_df = user_df.sort_values('Estimate_Score', ascending=False)
    # print(user_df.head(10))

    user_df_sum = Estimate_Score_sum(user_df, user_release_ratio_list)
    # user_df_sum = user_df_sum.sort_values('Estimate_Score', ascending=False)
    # print(user_df_sum.head(10))

    user_df_sum = Estimate_Score_sum2(user_df_sum, user_pop_ratio_list)
    # user_df_sum = user_df_sum.sort_values('Estimate_Score', ascending=False)
    # print(user_df_sum.head(10))

    user_df_sum = Estimate_Score_sum3(user_df_sum, user_language_ratio_list)
    user_df_sum = user_df_sum.sort_values('Estimate_Score', ascending=False)
    print(user_df_sum.head(10))

    return user_df, user_df_sum
test,test2 = userRec3(df,665,5,meta,drop_movie_list,reader,knn)
test,test2 = userRec3(df,665,5,meta,drop_movie_list,reader,svd)
test,test2 = userRec3(df,665,5,meta,drop_movie_list,reader,slope)
test,test2 = userRec3(df,665,5,meta,drop_movie_list,reader,nmf)

In [ ]:
test2.head(20)

,original_title,genres,release_date,popularity,original_language,Estimate_Score
11152,Crank,"[Action, Thriller, Crime]",2006-08-31,12.627593,en,6.348891
16351,The Return of the King,"[Animation, Family, Fantasy]",1980-05-11,1.800615,en,6.114000
6910,Birdman of Alcatraz,[Drama],1962-07-03,4.471515,en,6.093258
4905,Wuthering Heights,"[Drama, Romance]",1970-06-09,1.090793,en,6.058006
11560,Hannibal Rising,"[Crime, Drama, Thriller]",2007-02-06,2.159558,en,6.027443
278,Nell,"[Drama, Thriller]",1994-12-23,5.930957,en,6.004175
700,Dead Man,"[Drama, Fantasy, Western]",1995-12-23,6.505272,en,5.991708
11566,Lonely Hearts,"[Drama, Thriller, Crime, Romance]",2006-04-30,5.694413,en,5.984882
22094,End of the World,"[Fantasy, Thriller, Science Fiction]",1977-08-01,1.239669,en,5.959950
2283,Shakespeare in Love,"[Romance, History]",1998-12-11,13.813155,en,5.934520
